In [25]:
%run ../utilities/web_utilities.ipynb
%run ../utilities/database_utilities.ipynb
%run ../utilities/encoder_utilities.ipynb

In [26]:
write_mode = 'replace'
target_table = 'whl_team_list_by_season'

In [27]:
import pandas as pd
import sys
sys.path.append('../../../')

import settings as st

In [28]:
db_host = st.database['local']['host']
db_port = st.database['local']['port']
db_user = st.database['local']['user']
db_pass = st.database['local']['pass']
db_sys = st.database['local']['system']
db_db = st.database['local']['db']

In [29]:
key = "41b145a848f4bd67"

In [30]:
sql = "SELECT * FROM whl_season_list"

In [31]:
res = query_database_to_list(db_sys, db_user, db_pass, db_host, db_port, db_db, sql)

In [32]:
master_df = pd.DataFrame()

In [33]:
for r in res:
    season_id = r['season_id']
    url = "http://lscluster.hockeytech.com/feed/?feed=modulekit&view=teamsbyseason&key={0}&fmt=json&client_code=whl&lang=en&season_id={1}&league_code=&fmt=json".format(key, season_id)
    
    json_data = return_json(url)
    
    df = pd.DataFrame(json_data['SiteKit']['Teamsbyseason'])
    
    df['season_id'] = season_id
    df["division_id"] = pd.to_numeric(df["division_id"])
    df["id"] = pd.to_numeric(df["id"])
    
    master_df = master_df.append(df)

In [34]:
master_df = master_df.reset_index(drop=True)

In [36]:
if write_mode == 'append':
    index = get_table_new_id(db_sys, db_user, db_pass, db_host, db_port, db_db, target_table, 'index')
    master_df.insert(0, 'index', master_df.index + index)
else:
    master_df.insert(0, 'index', master_df.index)

In [37]:
write_df_to_database(master_df, target_table, db_sys, db_user, db_pass, db_host, db_port, db_db, write_mode, False)